In [1]:
#https://towardsdatascience.com/text-summarization-in-python-3f5a25418606

In [2]:
from nltk.cluster.util import cosine_distance
from nltk.corpus import stopwords
import numpy as np
import networkx as nx

In [241]:
import pandas as pd

data = pd.read_json('ExampleData2.json', orient='records')

In [267]:
#DEBUG
#data.text[0]

In [243]:
def getSentences(data3):
    #file = open(file_name, "r")
    filedata = data3
    #print(filedata)
    article = filedata.split(". ")
    #print(article)
    sentences = []
    for sentence in article:
        #print(sentence + "\n")
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
        #sentences.pop() 
        
    return sentences

In [244]:
#sentence100 = getSentences(data.text[0])

In [245]:
#sentence100

In [246]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [247]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
            
    return similarity_matrix

In [248]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samuelyoung/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [263]:
def generate_summary(file_name, top_n=2):
    
    stop_words = stopwords.words('english')
    summarize_text = []    
    
    # Step 1 - Read text and tokenize
    sentences =  getSentences(file_name)    
    
    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)    
    
    print(sentence_similarity_martix)
    
    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)    
    
    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)
    
    #print(ranked_sentence[i][1])
    
    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))    
    
    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))

In [264]:
generate_summary(data.text[0])

[[0.         0.39223227 0.29017165 0.21483446]
 [0.39223227 0.         0.32879797 0.30429031]
 [0.29017165 0.32879797 0.         0.2701351 ]
 [0.21483446 0.30429031 0.2701351  0.        ]]
Indexes of top ranked_sentence order are  [(0.2808037013724247, ['Built', 'on', 'Red', 'Hat®', 'OpenShift®', 'Container', 'Platform,', 'IBM', 'Cloud', 'Pak', 'for', 'Data', 'integrates', 'market-leading', 'IBM', 'Watson®', 'AI', 'technology', 'with', 'IBM', 'Hybrid', 'Data', 'Management', 'Platform,', 'DataOps,', 'and', 'governance', 'and', 'business', 'analytics', 'technologies']), (0.2489858176709867, ['IBM', 'Cloud', 'Pak™', 'for', 'Data', 'is', 'a', 'fully-integrated', 'data', 'and', 'AI', 'platform', 'that', 'modernizes', 'how', 'businesses', 'collect,', 'organize', 'and', 'analyze', 'data', 'and', 'infuse', 'AI', 'throughout', 'their', 'organizations']), (0.24732744158280123, ['Together,', 'these', 'capabilities', 'provide', 'the', 'information', 'architecture', 'for', 'AI', 'that', 'meets', 'y